In [1]:
# Publish pipeline

%%writefile training_pipeline/scoring.py

import json
import pickle
import numpy as np
import os

def init():
    global model, scaler
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "wine-quality-lr/1/model.pkl")
    print(model_path)
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    scaler_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "wine-quality-scaler/15/scaler.pkl")
    with open(scaler_path, "rb") as f:
        scaler = pickle.load(f)
    
    
def run(raw_data):
    data = np.array(json.loads(raw_data)["data"])
    prepped = scaler.transform(data)
    predictions = model.predict(prepped)
    return predictions.tolist()

 

UsageError: Line magic function `%%writefile` not found.


In [2]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies
service_env = Environment(name="service-env")
service_env.python.conda_dependencies = CondaDependencies.create(python_version="3.8",
                                                                            pip_packages=["numpy", "pandas",
                                                                                          "scikit-learn", "azureml-core",
                                                                                          "azureml-defaults", "azureml-pipeline"])


In [4]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.model import Model
from azureml.core import Workspace

model_inference_config = InferenceConfig(source_directory="training_pipeline",
                                        entry_script="scoring.py",
                                        environment=service_env)


ws = Workspace.from_config()
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)
model2 = Model(ws, name="wine-quality-lr")
model1 = Model(ws, name="wine-quality-scaler")
service = Model.deploy(ws, "wine-quality-aci", [model1, model2], model_inference_config, aci_config)
service.wait_for_deployment(show_output = True)
print(service.state)

service.get_logs()

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-27 14:28:39+00:00 Registering the environment.
2022-01-27 14:28:43+00:00 Use the existing image.
2022-01-27 14:28:43+00:00 Generating deployment configuration.
2022-01-27 14:28:45+00:00 Submitting deployment to compute..
2022-01-27 14:28:52+00:00 Checking the status of deployment wine-quality-aci..
2022-01-27 14:30:34+00:00 Checking the status of inference endpoint wine-quality-aci.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


'2022-01-27T14:30:20,689212300+00:00 - iot-server/run \n2022-01-27T14:30:20,690709100+00:00 - rsyslog/run \n2022-01-27T14:30:20,702757200+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\n2022-01-27T14:30:20,809522200+00:00 - nginx/run \nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (73)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 95\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2022-01-27T14:30:21,736970900+00:00 - iot-server/finish 1 0\n2022-01-27T14:30:21,743957700+00:00 - Exit code 1 is normal. Not restarting iot-server.\nSPARK_HOME not set. Skipping PySpark Initialization.\nInitializing logger\n2022-01-27 14:30:22,331 | root | INFO | Starting up app insights client\nlogging socket was found. logging is available.\nlogging socket was found. logging is available.\n2022-01-27 14:30:22,332 | root | INFO | Starting up request id generator\n2022-01-27 14:30:22,333

In [9]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
data = {"data": [[6.6, 0.45, 0.43, 7.2, 0.064, 31.0, 186.0, 0.9954, 3.12, 0.44, 9.4],
 [8.1, 0.3, 0.49, 12.3, 0.049, 50.0, 144.0, 0.9971, 3.09, 0.57, 10.2],
 [5.9, 0.27, 0.27, 9.0, 0.051, 43.0, 136.0, 0.9941, 3.25, 0.53, 10.7],
 [6.5, 0.28, 0.35, 9.8, 0.067, 61.0, 180.0, 0.9972, 3.15, 0.57, 9.0],
 [6.4, 0.27, 0.49, 7.3, 0.046, 53.0, 206.0, 0.9956, 3.24, 0.43, 9.2]]

}

body = str.encode(json.dumps(data))

url = 'http://a2a7dde7-a1c6-414a-ad27-350cc3ab3559.centralindia.azurecontainer.io/score'
api_key = '' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))


b'[0, 0, 0, 0, 0]'


In [10]:
service.run(input_data=json.dumps({"data": [[6.6, 0.45, 0.43, 7.2, 0.064, 31.0, 186.0, 0.9954, 3.12, 0.44, 9.4],
 [8.1, 0.3, 0.49, 12.3, 0.049, 50.0, 144.0, 0.9971, 3.09, 0.57, 10.2],
 [5.9, 0.27, 0.27, 9.0, 0.051, 43.0, 136.0, 0.9941, 3.25, 0.53, 10.7],
 [6.5, 0.28, 0.35, 9.8, 0.067, 61.0, 180.0, 0.9972, 3.15, 0.57, 9.0],
 [6.4, 0.27, 0.49, 7.3, 0.046, 53.0, 206.0, 0.9956, 3.24, 0.43, 9.2]]
}))

[0, 0, 0, 0, 0]